In [84]:
from use_translator import *

In [85]:
def create_query(disease_id):
    query_graph = {'nodes': {'disease': {'ids': [disease_id]}},
                   'edges': {}}
    pnode = 'disease'
    cnode = 'chemical'
    query_graph['nodes'][cnode] = {'categories': ['biolink:ChemicalEntity']}
    query_graph['edges']['t_edge'] = {'object': pnode, 'subject': cnode, 'predicates':['biolink:treats']}
    m = {'message':{'query_graph':query_graph}}
    return m

In [86]:
input_id = "MONDO:0004975"

In [87]:
tq = create_query(input_id)

In [88]:
aragorn_result = aragorn(tq)

aragorn returned in 0:05:42.325987s
Num Results: 3063


In [105]:
get_provenance(aragorn_result)

,QG Edge,Source,Count
0,t_edge,infores:aragorn,79639
1,t_edge,infores:biothings-explorer,51522
2,t_edge,infores:text-mining-provider-targeted,29222
3,t_edge,infores:chembl,23796
4,t_edge,infores:spoke,9469
...,...,...,...
7119,s2694,infores:aragorn-ranker-ara,1
7120,s2829,infores:aragorn-ranker-ara,1
7121,s2863,infores:aragorn-ranker-ara,1
7122,s2911,infores:aragorn-ranker-ara,1


In [89]:
def make_trapi(added_node, predicate, added_subject):
    query_graph = {'nodes':{'chemical':{'categories':['biolink:ChemicalEntity']},
                            'added_node': {'ids':[added_node]}},
                   'edges':{'x': {'subject': 'chemical', 'object':'added_node', 'predicates':[predicate]}}}
    return query_graph

def extract_query(knowledge_graph,result,did):
    #The set of original (pre coalesced) answers
    answers = set(x['id'] for x in result['node_bindings']['chemical'])
    #Node binding for AC'd addition
    nb = None
    for k in result['node_bindings']:
        if k.startswith('_n_ac'):
            nb = k
    if nb is None:
        return []
    #The actual node that got added:
    added_node = result['node_bindings'][nb][0]['id']
    if added_node == did:
        return []
    #Find the edges that got added:
    ebs = []
    for k in result['edge_bindings']:
        if k.startswith('_e_ac'):
            ebs.append(k)
    trapis = []
    for eb in ebs:
        #knowledge graph edges associated with this binding
        kg_ids = [x['id'] for x in result['edge_bindings'][eb]]
        preds = set([knowledge_graph['edges'][x]['predicate'] for x in kg_ids])
        subjects = set([knowledge_graph['edges'][x]['subject'] for x in kg_ids])
        objects = set([knowledge_graph['edges'][x]['object'] for x in kg_ids])
        if len(preds) > 1:
            print(preds)
            raise 'hell'
        predicate = preds.pop()
        if predicate in ['biolink:related_to','biolink:causes','biolink:contributes_to','biolink:contraindicated_for','biolink:biomarker_for']:
            continue
        added_subject = True
        if len(subjects) == 1:
            if not added_node in subjects:
                raise 'cain'
            added_subject = True
        elif len(objects)== 1:
            if not added_node in objects:
                raise 'the roof'
            added_subject = False
        else:
            raise 'your glass'
        trapis.append(make_trapi(added_node, predicate, added_subject))
    return trapis
        

In [90]:
enriched_results = aragorn_result['message']['results']

In [91]:
ts = []
for rule_n,er in enumerate(enriched_results):
    x = extract_query(aragorn_result['message']['knowledge_graph'],er,input_id)
    if len(x) > 0:
        #print(x)
        ts.extend(x)

In [92]:
all_results = []
for n,t in enumerate(ts):
    m = {'message': {'query_graph': t}}
    print(n,'/',len(ts))
    print(t)
    try:
        r = automat('covidkopkg',m)
        all_results.append(r)
    except e:
        print('blerk')

0 / 875
{'nodes': {'chemical': {'categories': ['biolink:ChemicalEntity']}, 'added_node': {'ids': ['NCBIGene:1576']}}, 'edges': {'x': {'subject': 'chemical', 'object': 'added_node', 'predicates': ['biolink:molecularly_interacts_with']}}}
automat/covidkopkg returned in 0:00:03.863366s
Num Results: 513
1 / 875
{'nodes': {'chemical': {'categories': ['biolink:ChemicalEntity']}, 'added_node': {'ids': ['NCBIGene:836']}}, 'edges': {'x': {'subject': 'chemical', 'object': 'added_node', 'predicates': ['biolink:activity_increased_by']}}}
automat/covidkopkg returned in 0:00:00.997205s
Num Results: 0
2 / 875
{'nodes': {'chemical': {'categories': ['biolink:ChemicalEntity']}, 'added_node': {'ids': ['MONDO:0001386']}}, 'edges': {'x': {'subject': 'chemical', 'object': 'added_node', 'predicates': ['biolink:treats']}}}
automat/covidkopkg returned in 0:00:01.823755s
Num Results: 0
3 / 875
{'nodes': {'chemical': {'categories': ['biolink:ChemicalEntity']}, 'added_node': {'ids': ['MONDO:0001152']}}, 'edges': 

automat/covidkopkg returned in 0:00:02.158197s
Num Results: 166
29 / 875
{'nodes': {'chemical': {'categories': ['biolink:ChemicalEntity']}, 'added_node': {'ids': ['MONDO:0005359']}}, 'edges': {'x': {'subject': 'chemical', 'object': 'added_node', 'predicates': ['biolink:ameliorates']}}}
automat/covidkopkg returned in 0:00:04.021907s
Num Results: 315
30 / 875
{'nodes': {'chemical': {'categories': ['biolink:ChemicalEntity']}, 'added_node': {'ids': ['MONDO:0005559']}}, 'edges': {'x': {'subject': 'chemical', 'object': 'added_node', 'predicates': ['biolink:treats']}}}
automat/covidkopkg returned in 0:00:01.872942s
Num Results: 90
31 / 875
{'nodes': {'chemical': {'categories': ['biolink:ChemicalEntity']}, 'added_node': {'ids': ['MONDO:0005567']}}, 'edges': {'x': {'subject': 'chemical', 'object': 'added_node', 'predicates': ['biolink:treats']}}}
automat/covidkopkg returned in 0:00:01.482929s
Num Results: 10
32 / 875
{'nodes': {'chemical': {'categories': ['biolink:ChemicalEntity']}, 'added_node

automat/covidkopkg returned in 0:00:01.141495s
Num Results: 0
57 / 875
{'nodes': {'chemical': {'categories': ['biolink:ChemicalEntity']}, 'added_node': {'ids': ['MONDO:0005044']}}, 'edges': {'x': {'subject': 'chemical', 'object': 'added_node', 'predicates': ['biolink:gene_associated_with_condition']}}}
automat/covidkopkg returned in 0:00:00.948838s
Num Results: 0
58 / 875
{'nodes': {'chemical': {'categories': ['biolink:ChemicalEntity']}, 'added_node': {'ids': ['MONDO:0005027']}}, 'edges': {'x': {'subject': 'chemical', 'object': 'added_node', 'predicates': ['biolink:gene_associated_with_condition']}}}
automat/covidkopkg returned in 0:00:00.970700s
Num Results: 0
59 / 875
{'nodes': {'chemical': {'categories': ['biolink:ChemicalEntity']}, 'added_node': {'ids': ['NCBIGene:3569']}}, 'edges': {'x': {'subject': 'chemical', 'object': 'added_node', 'predicates': ['biolink:interacts_with']}}}
automat/covidkopkg returned in 0:00:02.723930s
Num Results: 8
60 / 875
{'nodes': {'chemical': {'categori

automat/covidkopkg returned in 0:00:03.395858s
Num Results: 0
86 / 875
{'nodes': {'chemical': {'categories': ['biolink:ChemicalEntity']}, 'added_node': {'ids': ['GO:0051050']}}, 'edges': {'x': {'subject': 'chemical', 'object': 'added_node', 'predicates': ['biolink:has_part']}}}
automat/covidkopkg returned in 0:00:02.912073s
Num Results: 0
87 / 875
{'nodes': {'chemical': {'categories': ['biolink:ChemicalEntity']}, 'added_node': {'ids': ['GO:0032940']}}, 'edges': {'x': {'subject': 'chemical', 'object': 'added_node', 'predicates': ['biolink:has_part']}}}
automat/covidkopkg returned in 0:00:03.493173s
Num Results: 0
88 / 875
{'nodes': {'chemical': {'categories': ['biolink:ChemicalEntity']}, 'added_node': {'ids': ['MONDO:0005371']}}, 'edges': {'x': {'subject': 'chemical', 'object': 'added_node', 'predicates': ['biolink:genetic_association']}}}
automat/covidkopkg returned in 0:00:01.141503s
Num Results: 0
89 / 875
{'nodes': {'chemical': {'categories': ['biolink:ChemicalEntity']}, 'added_node

automat/covidkopkg returned in 0:00:02.678140s
Num Results: 0
115 / 875
{'nodes': {'chemical': {'categories': ['biolink:ChemicalEntity']}, 'added_node': {'ids': ['NCBIGene:2782']}}, 'edges': {'x': {'subject': 'chemical', 'object': 'added_node', 'predicates': ['biolink:interacts_with']}}}
automat/covidkopkg returned in 0:00:02.209150s
Num Results: 0
116 / 875
{'nodes': {'chemical': {'categories': ['biolink:ChemicalEntity']}, 'added_node': {'ids': ['NCBIGene:111']}}, 'edges': {'x': {'subject': 'chemical', 'object': 'added_node', 'predicates': ['biolink:interacts_with']}}}
automat/covidkopkg returned in 0:00:02.742693s
Num Results: 0
117 / 875
{'nodes': {'chemical': {'categories': ['biolink:ChemicalEntity']}, 'added_node': {'ids': ['NCBIGene:2793']}}, 'edges': {'x': {'subject': 'chemical', 'object': 'added_node', 'predicates': ['biolink:interacts_with']}}}
automat/covidkopkg returned in 0:00:02.136133s
Num Results: 0
118 / 875
{'nodes': {'chemical': {'categories': ['biolink:ChemicalEntity

...

automat/covidkopkg returned in 0:00:03.526130s
Num Results: 0
753 / 875
{'nodes': {'chemical': {'categories': ['biolink:ChemicalEntity']}, 'added_node': {'ids': ['GO:0048609']}}, 'edges': {'x': {'subject': 'chemical', 'object': 'added_node', 'predicates': ['biolink:has_part']}}}
automat/covidkopkg returned in 0:00:03.381428s
Num Results: 0
754 / 875
{'nodes': {'chemical': {'categories': ['biolink:ChemicalEntity']}, 'added_node': {'ids': ['GO:0007292']}}, 'edges': {'x': {'subject': 'chemical', 'object': 'added_node', 'predicates': ['biolink:has_part']}}}
automat/covidkopkg returned in 0:00:02.939363s
Num Results: 0
755 / 875
{'nodes': {'chemical': {'categories': ['biolink:ChemicalEntity']}, 'added_node': {'ids': ['GO:0008585']}}, 'edges': {'x': {'subject': 'chemical', 'object': 'added_node', 'predicates': ['biolink:has_part']}}}
automat/covidkopkg returned in 0:00:03.545400s
Num Results: 0
756 / 875
{'nodes': {'chemical': {'categories': ['biolink:ChemicalEntity']}, 'added_node': {'ids':

automat/covidkopkg returned in 0:00:02.634872s
Num Results: 0
782 / 875
{'nodes': {'chemical': {'categories': ['biolink:ChemicalEntity']}, 'added_node': {'ids': ['NCBIGene:259290']}}, 'edges': {'x': {'subject': 'chemical', 'object': 'added_node', 'predicates': ['biolink:interacts_with']}}}
automat/covidkopkg returned in 0:00:02.222601s
Num Results: 0
783 / 875
{'nodes': {'chemical': {'categories': ['biolink:ChemicalEntity']}, 'added_node': {'ids': ['NCBIGene:259293']}}, 'edges': {'x': {'subject': 'chemical', 'object': 'added_node', 'predicates': ['biolink:interacts_with']}}}
automat/covidkopkg returned in 0:00:02.754917s
Num Results: 0
784 / 875
{'nodes': {'chemical': {'categories': ['biolink:ChemicalEntity']}, 'added_node': {'ids': ['NCBIGene:259295']}}, 'edges': {'x': {'subject': 'chemical', 'object': 'added_node', 'predicates': ['biolink:interacts_with']}}}
automat/covidkopkg returned in 0:00:02.234931s
Num Results: 0
785 / 875
{'nodes': {'chemical': {'categories': ['biolink:Chemica

automat/covidkopkg returned in 0:00:04.144797s
Num Results: 0
811 / 875
{'nodes': {'chemical': {'categories': ['biolink:ChemicalEntity']}, 'added_node': {'ids': ['GO:1901653']}}, 'edges': {'x': {'subject': 'chemical', 'object': 'added_node', 'predicates': ['biolink:has_part']}}}
automat/covidkopkg returned in 0:00:03.557341s
Num Results: 0
812 / 875
{'nodes': {'chemical': {'categories': ['biolink:ChemicalEntity']}, 'added_node': {'ids': ['MONDO:0006176']}}, 'edges': {'x': {'subject': 'chemical', 'object': 'added_node', 'predicates': ['biolink:genetic_association']}}}
automat/covidkopkg returned in 0:00:01.147787s
Num Results: 0
813 / 875
{'nodes': {'chemical': {'categories': ['biolink:ChemicalEntity']}, 'added_node': {'ids': ['MONDO:0003197']}}, 'edges': {'x': {'subject': 'chemical', 'object': 'added_node', 'predicates': ['biolink:genetic_association']}}}
automat/covidkopkg returned in 0:00:01.159577s
Num Results: 0
814 / 875
{'nodes': {'chemical': {'categories': ['biolink:ChemicalEnti

automat/covidkopkg returned in 0:00:02.269579s
Num Results: 2
840 / 875
{'nodes': {'chemical': {'categories': ['biolink:ChemicalEntity']}, 'added_node': {'ids': ['NCBIGene:1909']}}, 'edges': {'x': {'subject': 'chemical', 'object': 'added_node', 'predicates': ['biolink:interacts_with']}}}
automat/covidkopkg returned in 0:00:02.584555s
Num Results: 8
841 / 875
{'nodes': {'chemical': {'categories': ['biolink:ChemicalEntity']}, 'added_node': {'ids': ['MONDO:0002492']}}, 'edges': {'x': {'subject': 'chemical', 'object': 'added_node', 'predicates': ['biolink:genetic_association']}}}
automat/covidkopkg returned in 0:00:01.317177s
Num Results: 0
842 / 875
{'nodes': {'chemical': {'categories': ['biolink:ChemicalEntity']}, 'added_node': {'ids': ['NCBIGene:481']}}, 'edges': {'x': {'subject': 'chemical', 'object': 'added_node', 'predicates': ['biolink:entity_regulates_entity']}}}
automat/covidkopkg returned in 0:00:01.835683s
Num Results: 37
843 / 875
{'nodes': {'chemical': {'categories': ['biolink

automat/covidkopkg returned in 0:00:02.953405s
Num Results: 0
869 / 875
{'nodes': {'chemical': {'categories': ['biolink:ChemicalEntity']}, 'added_node': {'ids': ['GO:0050818']}}, 'edges': {'x': {'subject': 'chemical', 'object': 'added_node', 'predicates': ['biolink:has_part']}}}
automat/covidkopkg returned in 0:00:03.529897s
Num Results: 0
870 / 875
{'nodes': {'chemical': {'categories': ['biolink:ChemicalEntity']}, 'added_node': {'ids': ['GO:0061041']}}, 'edges': {'x': {'subject': 'chemical', 'object': 'added_node', 'predicates': ['biolink:has_part']}}}
automat/covidkopkg returned in 0:00:03.437715s
Num Results: 0
871 / 875
{'nodes': {'chemical': {'categories': ['biolink:ChemicalEntity']}, 'added_node': {'ids': ['GO:0061045']}}, 'edges': {'x': {'subject': 'chemical', 'object': 'added_node', 'predicates': ['biolink:has_part']}}}
automat/covidkopkg returned in 0:00:02.938363s
Num Results: 0
872 / 875
{'nodes': {'chemical': {'categories': ['biolink:ChemicalEntity']}, 'added_node': {'ids':

In [104]:
printjson(all_results[0]['message']['results'][0])

{
    "node_bindings": {
        "chemical": [
            {
                "id": "PUBCHEM.COMPOUND:5324346"
            }
        ],
        "added_node": [
            {
                "id": "NCBIGene:1576",
                "qnode_id": "NCBIGene:1576"
            }
        ]
    },
    "edge_bindings": {
        "x": [
            {
                "id": "c5409f8cbedb6476e29668bef5135188",
                "attributes": null
            }
        ]
    },
    "score": null
}


In [120]:
def merge_answer(results,qnode_ids,p=False):
    #We are going rename most of the node and edge bindings.  But, we want to preserve bindings to things in the
    # original query.  For the current creative one-hop, that is just nodes b/c we are replacing the one edge.
    #The original qnode bindings are the same in all results by construction
    mergedresult = {'node_bindings': { q: results[0]['node_bindings'][q] for q in qnode_ids},
                    'edge_bindings': {}}
    #if 'PUBCHEM.COMPOUND:6623' in qnode_ids:
    #    print('ooo')
    if p:
        for r in results:
            print('x')
    for bindingtype in ['node', 'edge']:
        bound_things = set()
        for result in results:
            # the gross part here is dealing with the lists in the values of the bindings.
            for key,binding in result[f'{bindingtype}_bindings'].items():
                if key in qnode_ids:
                    continue
                bound = frozenset([x['id'] for x in binding])
                if bound not in bound_things:
                    #found a binding to add
                    n = f'_dummy_{bindingtype}_{len(bound_things)}'
                    mergedresult[f'{bindingtype}_bindings'][n] = binding
                    bound_things.add(bound)
    if p:
        print(mergedresult['node_bindings'])
    return mergedresult

def add_bindings(result,more_bindings):
    current_bindings = set([nb[0]['id'] for nb in result['node_bindings'].values()])
    for b,node_id in more_bindings.items():
        if node_id not in current_bindings:
            result['node_bindings'][b] = [{'id':node_id}]

from copy import deepcopy
#TODO move into operations? Make a translator op out of this
def merge_results_by_node(result_message, merge_qnode, extra_nodes):
    """This assumes a single result message, with a single merged KG.  The goal is to take all results that share a
    binding for merge_qnode and combine them into a single result.
    Assumes that the results are not scored."""
    #This is relatively straightforward: group all the results by the merge_qnode
    # for each one, the only complication is in the keys for the "dummy" bindings.
    original_results = result_message['message']['results']
    #original_qnodes = result_message['message']['query_graph']['nodes'].keys()
    #hackhack
    original_qnodes = set(['chemical'])
    #print(original_qnodes)
    #group results
    grouped_results = defaultdict(list)
    for result in original_results:
        answer = result['node_bindings'][merge_qnode]
        bound = frozenset([x['id'] for x in answer])
        grouped_results[bound].append(result)
    #TODO : I'm sure there's a better way to handle this with asyncio
    new_results = []
    for r in grouped_results:
        p = False
        x = merge_answer(grouped_results[r],original_qnodes,p)
        add_bindings(x,extra_nodes)
        new_results.append(x)
    return_message = deepcopy(result_message)
    return_message['message']['results'] = new_results
    return return_message


In [95]:
from reasoner_pydantic import KnowledgeGraph

In [96]:
pydantic_kgraph = KnowledgeGraph.parse_obj({"nodes":{}, "edges":{}})
for rm in all_results:
    if len(rm['message']['results']) == 0:
        continue
    pydantic_kgraph.update(KnowledgeGraph.parse_obj(rm['message']['knowledge_graph']))
tresult = all_results[0]
tresult['message']['knowledge_graph'] = pydantic_kgraph.dict()

In [121]:
#tresult['message']['query_graph'] = deepcopy(input_message['message']['query_graph'])
for rm in all_results[1:]:
    tresult['message']['results'].extend( rm['message']['results'])
mergedresults = merge_results_by_node(tresult,'chemical',{'original_input':'MONDO:0004975'})

In [122]:
mergedresults['workflow']=[{'id':'overlay_connect_knodes'}, {'id':'score'}]
f = aragorn(mergedresults)

aragorn returned in 0:03:47.760106s
Num Results: 3331


In [124]:
del f['message']['query_graph']['nodes']['added_node']
f['message']['query_graph']['edges']={}

In [112]:
for nr,result in enumerate(f['message']['results']):
    if result['node_bindings']['chemical'][0]['id'] == 'PUBCHEM.COMPOUND:5324346':
        print(nr)
        printjson(result)

3023
{
    "node_bindings": {
        "chemical": [
            {
                "id": "PUBCHEM.COMPOUND:5324346"
            }
        ],
        "_dummy_node_0": [
            {
                "id": "NCBIGene:1576",
                "qnode_id": "NCBIGene:1576"
            }
        ],
        "_dummy_node_1": [
            {
                "id": "NCBIGene:5243",
                "qnode_id": "NCBIGene:5243"
            }
        ],
        "_dummy_node_2": [
            {
                "id": "NCBIGene:1565",
                "qnode_id": "NCBIGene:1565"
            }
        ],
        "_dummy_node_3": [
            {
                "id": "NCBIGene:1559",
                "qnode_id": "NCBIGene:1559"
            }
        ],
        "_dummy_node_4": [
            {
                "id": "NCBIGene:1557",
                "qnode_id": "NCBIGene:1557"
            }
        ],
        "_dummy_node_5": [
            {
                "id": "NCBIGene:1544",
                "qnode_id": "NCBIGe

In [125]:
with open('ad_enrich.json','w') as outf:
    json.dump(f,outf,indent=4)

In [74]:
f['workflow'] = [ {'id': 'filter_results_top_n', 'parameters': {'max_results': 1}}, {'id': 'filter_kgraph_orphans'}]

In [75]:
g = aragorn(f)

aragorn returned in 0:00:19.188855s
Num Results: 1


In [76]:
#h = deepcopy(g)
#h['message']['results'] = g['message']['results'][:1]
with open('ad_enrich.json','w') as outf:
    json.dump(g,outf,indent=4)

In [80]:
printjson(g)

{
    "message": {
        "query_graph": {
            "nodes": {
                "chemical": {
                    "ids": null,
                    "categories": [
                        "biolink:ChemicalEntity"
                    ],
                    "is_set": false,
                    "constraints": []
                },
                "added_node": {
                    "ids": [
                        "NCBIGene:1576"
                    ],
                    "categories": null,
                    "is_set": false,
                    "constraints": []
                }
            },
            "edges": {
                "x": {
                    "subject": "chemical",
                    "object": "added_node",
                    "predicates": [
                        "biolink:molecularly_interacts_with"
                    ],
                    "constraints": []
                }
            }
        },
        "knowledge_graph": {
            "nodes": {
          

In [79]:
for nodeid,node in g['message']['knowledge_graph']['nodes'].items():
    node['attributes'] = None

In [48]:
len(g['message']['knowledge_graph']['edges'])

301

In [71]:
all_results = []
for t in [{'nodes': {'chemical': {'categories': ['biolink:ChemicalEntity']}, 'added_node': {'ids': ['NCBIGene:259294']}}, 'edges': {'x': {'subject': 'chemical', 'object': 'added_node', 'predicates': ['biolink:interacts_with']}}}]:
    m = {'message': {'query_graph': t}}
    print(m)
    try:
        r = automat('covidkopkg',m)
        all_results.append(r)
    except e:
        print('blerk')

{'message': {'query_graph': {'nodes': {'chemical': {'categories': ['biolink:ChemicalEntity']}, 'added_node': {'ids': ['NCBIGene:259294']}}, 'edges': {'x': {'subject': 'chemical', 'object': 'added_node', 'predicates': ['biolink:interacts_with']}}}}}
automat/covidkopkg returned in 0:00:02.626203s
Num Results: 0
